# IPL Powerplay Score Prediction
![IPL banner](https://cricketaddictor.gumlet.io/wp-content/uploads/2021/02/153757448_173638941016448_6980867142752435675_n.jpg?compress=true&quality=80&w=1920&dpr=2.6)

It's COVID lockdown time. Everyone is worried about the increasing COVID cases, Work From Home (WFH) environment. No way to go out, no parties, no outing, no vacation plans, no gatherings ...... 

Government announces Lockdown : 1.0, 2.0, 3.0, 4.0 .....

At that time every cricket fan had a question. 
> **"Do new Govt. rules in lockdown 4.0 pave the way for IPL 2020 ????"**



Finally IPL 2020 happend. 
* **Even pandamic situation can't derail the IPL juggernaut.**
* IPL 20202 earned revenue of : 4000 crores with 
  *  35% reduced cost and 
  *  25% increase in viewership. 

As a cricket fan I watch all the matches, during that time I observed that **"Powerplay plays a mojor role"** which is very Important in Team score.

## What is powerplay in IPL & it's Importance???
* In summary "powerplay" means fielding restrictions in 1<sup>st</sup> 6-overs.
  * means, only 2-fielders can stay outside the Inner-circle
  * After powerplay, upto 5-fielders can stay outside inner circle & 4-fielders must remain inside the inner circle.
* Which makes the **batting compatively easy.**
* Also **it's a trap for the batsmen**, as this will get them to take into a risk and loose their wickets in 1<sup>st</sup> 6-overs

## Effect of "Powerplay" in IPL matches??
* Powerplay overs are considered as pillars of any teams  victory.
* 75% - of winning chance of the team depends on the Powerplay score.
* So, every team's expectation from the top 3-batsmen is **"START THE INNINGS BIG"**

In this blog post I am going to predict the score of an IPL match at the end of powerplay.

In [1]:
import requests,zipfile,io,json,shutil
from pathlib import Path

#from sklearn.preprocessing import LabelEncoder
from collections import OrderedDict

import pandas as pd
import seaborn as sns
import numpy as np

import torch 

from matplotlib import pyplot as plt
%matplotlib inline

# **Variables & Hyper Params Declaration:**

In [2]:
epochs      = 100
lr          = 0.1
split_ratio = (70.0,30.0)
train_bs    = 32   # Then test_bs = 2*train_bs
pltName     = 'v2_test15.png'
modelPTH    = pltName.split('.')[0]+'_model.pt'

seed = 143
torch.manual_seed(seed)

In [3]:
dataset_path = Path(Path.cwd(),'dataset')
snippets_path = Path(Path.cwd(),'snippets')
filename = 'all_matches.csv'
recordedMetrics = 'Recorded_Metrics.csv'

json_path = Path.joinpath(dataset_path, 'label_encode.json')

save_files = False
record = True

# **1.Loading the Data Set:** 

Dataset will be available at : https://cricsheet.org/downloads/ipl_male_csv.zip

In [4]:
def getCsvFile(url="https://cricsheet.org/downloads/ipl_male_csv2.zip"):
    res = requests.get(url, stream=True)
    if res.status_code == 200:
        print('### Downloading the CSV file')
        z = zipfile.ZipFile(io.BytesIO(res.content))
        if filename in z.namelist():
            z.extract(filename, dataset_path)
            print('### Extracted %s file' % filename)
        else:
            print('### %s : File not found in ZIP Artifact' % filename)

def downloadDataset():
    if not dataset_path.exists():
        Path.mkdir(dataset_path)
        print('### Created Dataset folder')
        getCsvFile()
    elif dataset_path.exists():
        files = [file for file in dataset_path.iterdir() if file.name ==
                 'all_matches.csv']
        if len(files) == 0:
            getCsvFile()
        else:
            print('### File already extracted in given path')
downloadDataset()

### File already extracted in given path


# **2. Data Frames:** 

In [5]:
csv_file=Path.joinpath(dataset_path, filename)

df = pd.read_csv(csv_file,parse_dates=['start_date'],low_memory=False)
df_parsed = df.copy(deep=True)
print(df.shape)
df.head()


(200664, 22)


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **3. Exploratory Data Analysis (EDA) / Data Processing:**
## 3.1: Identify "null" values in Dataset:

In [6]:
df.isnull().sum()

match_id                       0
season                         0
start_date                     0
venue                          0
innings                        0
ball                           0
batting_team                   0
bowling_team                   0
striker                        0
non_striker                    0
bowler                         0
runs_off_bat                   0
extras                         0
wides                     194583
noballs                   199854
byes                      200135
legbyes                   197460
penalty                   200662
wicket_type               190799
player_dismissed          190799
other_wicket_type         200664
other_player_dismissed    200664
dtype: int64

## 3.2: Cleaning Steps:
1. Here columns 'season' and 'start_date'  are not needed for our prediction. ```So we can drop the columns 'season' and 'start_date' from the dataset.```
2. Delete Non-existing teams : 
```
       'Kochi Tuskers Kerala' 'Pune Warriors','Rising Pune Supergiants', 'Rising Pune Supergiant','Gujarat Lions'
```
3. Replace the old team names with new team name:
```
      'Delhi Daredevils'  --> 'Delhi Capitals'
      'Deccan Chargers'   --> 'Sunrisers Hyderabad'
      'Punjab Kings'      --> 'Kings XI Punjab'
```

4. Correct the venue column with unique names. In this dataset same stadium is being represented as in multiple ways. So identify those and rename.
```
      ['M Chinnaswamy Stadium', 'M.Chinnaswamy Stadium']
      ['Brabourne Stadium', 'Brabourne Stadium, Mumbai']
      ['Punjab Cricket Association Stadium, Mohali', 'Punjab Cricket Association IS Bindra Stadium, Mohali', 'Punjab Cricket Association IS Bindra Stadium']
      ['Wankhede Stadium', 'Wankhede Stadium, Mumbai']
      ['Rajiv Gandhi International Stadium, Uppal', 'Rajiv Gandhi International Stadium']
      ['MA Chidambaram Stadium, Chepauk','MA Chidambaram Stadium',      'MA Chidambaram Stadium, Chepauk, Chennai']
```

5. Rename the column names (for easy coding)
```
      'striker'     --> 'batsmen'
      'non-striker' --> 'batsmen_nonstriker'  (This column is not required)
      'bowler'      --> 'bowlers'
```
      
6. Create a columns "Total_score" : which reflects the runs through bat and extra runs through wides,byes,noballs,legbyes...etc.  
Hence we can drop columsn ```['wides', 'noballs', 'byes', 'legbyes', 'penalty', 'wicket_type','other_wicket_type', 'other_player_dismissed']```

In [7]:
df_parsed.drop(columns=['wides', 'noballs', 'byes', 'legbyes', 'penalty', 'wicket_type','other_wicket_type', 'other_player_dismissed','season','start_date'],axis=1,inplace=True)
df_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200664 entries, 0 to 200663
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   match_id          200664 non-null  int64  
 1   venue             200664 non-null  object 
 2   innings           200664 non-null  int64  
 3   ball              200664 non-null  float64
 4   batting_team      200664 non-null  object 
 5   bowling_team      200664 non-null  object 
 6   striker           200664 non-null  object 
 7   non_striker       200664 non-null  object 
 8   bowler            200664 non-null  object 
 9   runs_off_bat      200664 non-null  int64  
 10  extras            200664 non-null  int64  
 11  player_dismissed  9865 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 18.4+ MB


In [8]:
non_exist_teams = ['Kochi Tuskers Kerala',
                    'Pune Warriors',
                    'Rising Pune Supergiants',
                    'Rising Pune Supergiant',
                    'Gujarat Lions']
mask_bat_team = df_parsed['batting_team'].isin(non_exist_teams)
mask_bow_team = df_parsed['bowling_team'].isin(non_exist_teams)
df_parsed = df_parsed[~mask_bat_team]
df_parsed = df_parsed[~mask_bow_team]

<ipython-input-8-567adafd9830>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_parsed = df_parsed[~mask_bow_team]


In [9]:
df_parsed.loc[df_parsed.batting_team ==
                'Delhi Daredevils', 'batting_team'] = 'Delhi Capitals'
df_parsed.loc[df_parsed.batting_team == 'Deccan Chargers',
                'batting_team'] = 'Sunrisers Hyderabad'
df_parsed.loc[df_parsed.batting_team ==
                'Punjab Kings', 'batting_team'] = 'Kings XI Punjab'

df_parsed.loc[df_parsed.bowling_team ==
                'Delhi Daredevils', 'bowling_team'] = 'Delhi Capitals'
df_parsed.loc[df_parsed.bowling_team == 'Deccan Chargers',
                'bowling_team'] = 'Sunrisers Hyderabad'
df_parsed.loc[df_parsed.bowling_team ==
                'Punjab Kings', 'bowling_team'] = 'Kings XI Punjab'



In [10]:
df_parsed.venue.unique()

array(['M Chinnaswamy Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Feroz Shah Kotla',
       'Eden Gardens', 'Wankhede Stadium', 'Sawai Mansingh Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'MA Chidambaram Stadium, Chepauk', 'Dr DY Patil Sports Academy',
       'Newlands', "St George's Park", 'Kingsmead', 'SuperSport Park',
       'Buffalo Park', 'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium',
       'Sardar Patel Stadium, Motera', 'Barabati Stadium',
       'Brabourne Stadium, Mumbai',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sheikh Zayed Stadium',
       'Sharjah Cricket Stadium', 'Dubai International Cricket Stadium',
      

In [11]:
list(filter(lambda x : 'chidam' in x.lower(),list(df_parsed.venue.unique())))
    

['MA Chidambaram Stadium, Chepauk',
 'MA Chidambaram Stadium',
 'MA Chidambaram Stadium, Chepauk, Chennai']

In [12]:
df_parsed.loc[df_parsed.venue == 'M.Chinnaswamy Stadium',
                'venue'] = 'M Chinnaswamy Stadium'
df_parsed.loc[df_parsed.venue == 'Brabourne Stadium, Mumbai',
                'venue'] = 'Brabourne Stadium'
df_parsed.loc[df_parsed.venue == 'Punjab Cricket Association IS Bindra Stadium, Mohali',
                'venue'] = 'Punjab Cricket Association Stadium'
df_parsed.loc[df_parsed.venue == 'Punjab Cricket Association IS Bindra Stadium',
                'venue'] = 'Punjab Cricket Association Stadium'
df_parsed.loc[df_parsed.venue == 'Wankhede Stadium, Mumbai',
                'venue'] = 'Wankhede Stadium'
df_parsed.loc[df_parsed.venue == 'Rajiv Gandhi International Stadium, Uppal',
                'venue'] = 'Rajiv Gandhi International Stadium'
df_parsed.loc[df_parsed.venue == 'MA Chidambaram Stadium, Chepauk',
                'venue'] = 'MA Chidambaram Stadium'
df_parsed.loc[df_parsed.venue == 'MA Chidambaram Stadium, Chepauk, Chennai',
                'venue'] = 'MA Chidambaram Stadium'
df_parsed.loc[df_parsed.venue == 'Arun Jaitley Stadium','venue'] = 'Arun Jaitley Stadium, Delhi'
df_parsed.loc[df_parsed.venue == 'Punjab Cricket Association Stadium, Mohali','venue'] = 'Punjab Cricket Association Stadium'


In [13]:
df_parsed = df_parsed.rename(columns={
                                 'striker': 'batsmen', 'non_striker': 'batsmen_non_striker', 'bowler': 'bowlers'})

In [14]:
df_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173645 entries, 0 to 200663
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   match_id             173645 non-null  int64  
 1   venue                173645 non-null  object 
 2   innings              173645 non-null  int64  
 3   ball                 173645 non-null  float64
 4   batting_team         173645 non-null  object 
 5   bowling_team         173645 non-null  object 
 6   batsmen              173645 non-null  object 
 7   batsmen_non_striker  173645 non-null  object 
 8   bowlers              173645 non-null  object 
 9   runs_off_bat         173645 non-null  int64  
 10  extras               173645 non-null  int64  
 11  player_dismissed     8541 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 17.2+ MB


In [15]:
df_parsed['Total_score'] = df_parsed.runs_off_bat + df_parsed.extras

In [16]:
df_parsed.head()

,match_id,venue,innings,ball,batting_team,bowling_team,batsmen,batsmen_non_striker,bowlers,runs_off_bat,extras,player_dismissed,Total_score
0,335982,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,1
1,335982,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,0
2,335982,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,NaN,1
3,335982,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,0
4,335982,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,0


## 3.3: Select required Data:
   Here as we are predicting the IPL match **"Powerplay"** score, we can drop the rest of the balls and run detail through, 
   * by selecting the 1<sup>st</sup> 6-Overs details and  
   * with Innings (1 and 2) : as there is no 3<sup>rd</sup> or 4<sup>th</sup> innings happen in powerplay


In [17]:
df_parsed = df_parsed[(df_parsed.ball < 6.0) & (df_parsed.innings < 3)]

In [18]:
if save_files:
    df_parsed.to_csv(Path.joinpath(dataset_path, '1_parseNcleaned_Dataset.csv'), index=False)

print('### Total {} : venue details present '.format(
    len(df_parsed.venue.unique())))
print('### Total {}  : Batting teams are there'.format(
    len(df_parsed.batting_team.unique())))
print('### Total {}  : Bowlling teams are there'.format(
    len(df_parsed.bowling_team.unique())))
print('### Batting teams are : {}'.format(df_parsed.batting_team.unique()))
print('### Bowling teams are : {}'.format(df_parsed.bowling_team.unique()))
print('### Shape of data frame after initial cleanup :{}'.format(df_parsed.shape))


### Total 33 : venue details present 
### Total 8  : Batting teams are there
### Total 8  : Bowlling teams are there
### Batting teams are : ['Kolkata Knight Riders' 'Royal Challengers Bangalore'
 'Chennai Super Kings' 'Kings XI Punjab' 'Rajasthan Royals'
 'Delhi Capitals' 'Sunrisers Hyderabad' 'Mumbai Indians']
### Bowling teams are : ['Royal Challengers Bangalore' 'Kolkata Knight Riders' 'Kings XI Punjab'
 'Chennai Super Kings' 'Delhi Capitals' 'Rajasthan Royals'
 'Sunrisers Hyderabad' 'Mumbai Indians']
### Shape of data frame after initial cleanup :(54525, 13)


## **3.4. Identify "No Result" matches:**
* In our data few matches are not conducted and declared as "no result" may be due to rain... some technical problems.
* Either match result is declared by DCB method, we can't predict the score.

Which may lead to outliers, to our model. Because If team played only 3 overs, then score minimum value effects which leads to modify the following "Normalization" process. So better to drop these data in our prediction.


In [19]:
obj = df_parsed.query('ball<6.0&innings<3').groupby(['match_id','venue','innings','batting_team','bowling_team'])

for key,val in obj:
    if val['ball'].count()<36:
        df_parsed.drop(labels=val.index,axis=0,inplace=True)

In [20]:
Total_score_min_max = (float(df_parsed.groupby(['match_id','venue','innings','batting_team'])['Total_score'].sum().min()),(df_parsed.groupby(['match_id','innings'])['Total_score'].sum().max()))

Total_score_min_max

(14.0, 105)

In [21]:
print(df_parsed.shape)

df_parsed.head()

(54370, 13)


,match_id,venue,innings,ball,batting_team,bowling_team,batsmen,batsmen_non_striker,bowlers,runs_off_bat,extras,player_dismissed,Total_score
0,335982,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,1
1,335982,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,0
2,335982,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,NaN,1
3,335982,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,0
4,335982,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,0


# **4. Encoding:** 

Dataframe column values of Dtype Object:
Till now as a initial step in step-2,
* We cleaned our dataset with all the "null" values and 
* filtered the columns/rows data (which are not used for prediction) and
* Added the required column with values (like total_score) to make the dataset clean.

As you can see above, our cleaned dataset is having 13 columns of multiple Dtype like int64 and float64 and object.

So next what I am going to do is trying to convert all these multiple Dtypes into a single Dtype, to train my model.

## 4.1:  Encode "batsmen" and "bowlers" column values:
   * Here we can see, few players who can bat as well as bowl.
    Means same player will be listed as a batsmen and as well as bowler.
    * So to make the prediction properly, I am creating one Dataframe with all the players name it as "players_df", which I use for encoding the players with some value to identify.
* For inference going further, I create a dictionary with all these encoded values 


In [22]:
players_df = pd.DataFrame(np.append(
                            df_parsed.batsmen.unique(), df_parsed.bowlers.unique()),
                            columns=['Players']
                        )
players_df.head()

,Players
0,SC Ganguly
1,BB McCullum
2,RT Ponting
3,R Dravid
4,W Jaffer


In [23]:
label_encode_dict = {}

#le = LabelEncoder()
#le.fit(players_df.Players)
#Players_e = le.transform(players_df.Players)
#Players_e_inv = le.inverse_transform(Players_e)

dct = dict(enumerate(players_df.Players.astype('category').cat.categories))
label_encode_dict['Players'] = dict(zip(dct.values(),dct.keys()))

#label_encode_dict

## 4.2: Encode "vennue" and "batting_team" and "bowling_team" column values:

In [24]:
#df_parsed.select_dtypes('object').columns
for col in ['venue', 'batting_team', 'bowling_team']:
    dct = dict(enumerate(df_parsed[col].astype('category').cat.categories))
    label_encode_dict[col] = dict(zip(dct.values(),dct.keys()))

label_encode_dict.keys()


dict_keys(['Players', 'venue', 'batting_team', 'bowling_team'])

In [25]:
label_encode_dict['batting_team'] == label_encode_dict['bowling_team']

True

## 4.3: Save the encoded values to a json file

In [26]:
label_encode_dict['Total_score_min'] = float(Total_score_min_max[0])
label_encode_dict['Total_score_max'] = float(Total_score_min_max[1])
label_encode_dict['Total_score_min'],label_encode_dict['Total_score_max']


(14.0, 105.0)

In [27]:
with open(json_path, 'w') as f:
    json.dump(label_encode_dict, f)
    

In [28]:
df_parsed.head

<bound method NDFrame.head of         match_id                             venue  innings  ball  \
0         335982             M Chinnaswamy Stadium        1   0.1   
1         335982             M Chinnaswamy Stadium        1   0.2   
2         335982             M Chinnaswamy Stadium        1   0.3   
3         335982             M Chinnaswamy Stadium        1   0.4   
4         335982             M Chinnaswamy Stadium        1   0.5   
...          ...                               ...      ...   ...   
200587   1254086  Narendra Modi Stadium, Ahmedabad        2   5.2   
200588   1254086  Narendra Modi Stadium, Ahmedabad        2   5.3   
200589   1254086  Narendra Modi Stadium, Ahmedabad        2   5.4   
200590   1254086  Narendra Modi Stadium, Ahmedabad        2   5.5   
200591   1254086  Narendra Modi Stadium, Ahmedabad        2   5.6   

                 batting_team                 bowling_team      batsmen  \
0       Kolkata Knight Riders  Royal Challengers Bangalore   SC Ga

## 4.4: Format the Dataset:
In this step I am trying to club the all rows with respect to matchID and Innings (as match ID is unique way to identify a particular match and Innings to identify who bat first).

Based on these two details, 
* grab all the batsmen and bowler details who batted and bowled in 1<sup>st</sup> 6-overs
* Calculate the total score (runs through bat + extra runs)
* How many players dismissed in 1st 6-overs


In [29]:
print('### Shape of Dataframe before format_data : {}'.format(df_parsed.shape))
Runs_off_Bat_6_overs = df_parsed.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['runs_off_bat'].sum()
Extras_6_overs = df_parsed.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['extras'].sum()
TotalScore_6_overs = df_parsed.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['Total_score'].sum()

Total_WktsDown = df_parsed.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['player_dismissed'].count()


### Shape of Dataframe before format_data : (54370, 13)


In [30]:
bat_df = df_parsed.groupby(['match_id', 'venue', 'innings','batting_team', 'bowling_team'])['batsmen'].apply(list)
bow_df = df_parsed.groupby(['match_id', 'venue', 'innings','batting_team', 'bowling_team'])['bowlers'].apply(list)

now concat all these formated data into a single dataframe

In [31]:
df_parsed.shape

(54370, 13)

In [32]:
df_parsed = pd.DataFrame(pd.concat([bat_df, bow_df, Runs_off_Bat_6_overs, Extras_6_overs, TotalScore_6_overs, Total_WktsDown],axis=1)).reset_index()

if save_files:
    df_parsed.to_csv(Path.joinpath(dataset_path,'2_powerplay_summary.csv'),index=False)

## 4.5 : Align the batsmen and bowlers details in to a separate column

In above formated dataset, we got list of batsmen and bowlers details who batted and bowled in 6-overs.

Now we have to arrange these batsmen into a separate columns,
```
 * say bat1,bat2,bat3,bat4....bat10
 * say bow1,bow2,bow3.....bow6
```

Here I selected only 10-batsmen (as we have only 10-wickets), and 6-bowlers (can bowl in 6-overs) because in 6-overs this is only possible.

```
For proper prediction the order of batsmen and bowlers given the dataset matters. So we need to keep the order :
* batsmen : who batted 1st,2nd 3rd and 4th ... wicket same
* bowler  : who bowled in 1st,2nd,3rd,4th,5th and 6th overs same
```

## 4.6: Create a batsmen and bowlers dummy dataframe:
Inorder to keep track of the order same, so 1st I am going to create a dummy dataframe 
* with 10-batsmen with column names [bat1,bat2,.....bat9,bat10]
* with 6-bowlers with column names [bow1,bow2,bow3,bow4,bow5,bow6]

In [33]:
df_parsed.shape

(1453, 11)

In [34]:
bat  = pd.DataFrame(np.zeros((df_parsed.shape[0],10),dtype=float),columns=['bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10'])
bowl = pd.DataFrame(np.zeros((df_parsed.shape[0],6),dtype=float),columns=['bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6'])

columns = ['bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10','bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6']
df_bat_bow = pd.concat([bat,bowl],axis=1)
df_bat_bow



,bat1,bat2,bat3,bat4,bat5,bat6,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1451,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df_parsed =  pd.concat([df_parsed, df_bat_bow], axis=1)
if save_files:
    df_parsed.to_csv(Path.joinpath(dataset_path, '3_create_batsmen_bowler_df.csv'),index=False)

In [36]:
df_parsed.head(3)

,match_id,venue,innings,batting_team,bowling_team,batsmen,bowlers,runs_off_bat,extras,Total_score,...,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,"[SC Ganguly, BB McCullum, BB McCullum, BB McCu...","[P Kumar, P Kumar, P Kumar, P Kumar, P Kumar, ...",51,10,61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,335982,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,"[R Dravid, W Jaffer, W Jaffer, W Jaffer, R Dra...","[AB Dinda, AB Dinda, AB Dinda, AB Dinda, AB Di...",19,7,26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,335983,Punjab Cricket Association Stadium,1,Chennai Super Kings,Kings XI Punjab,"[PA Patel, PA Patel, PA Patel, ML Hayden, ML H...","[B Lee, B Lee, B Lee, B Lee, B Lee, B Lee, S S...",50,3,53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4.7: Update the batsmen list of elements into each column in the same order:

Means, data which is in list of elements in each matchID and innings into corresponding individual batsmen columns in the same order. 
```
Example, here in below list 1st batsmen is bat1 -> SC Ganguly, 2nd is bat2 -> BB McCullum, 3rd is bat3 -> RT Poting ...etc like 

"['SC Ganguly', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'SC Ganguly', 'SC Ganguly', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'SC Ganguly', 'SC Ganguly', 'BB McCullum', 'SC Ganguly', 'SC Ganguly', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'BB McCullum', 'SC Ganguly', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'Mohammad Hafeez', 'Mohammad Hafeez', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'Mohammad Hafeez', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum']"

```


In [37]:
df_parsed['batsmen'] = df_parsed.batsmen.map(OrderedDict.fromkeys).apply(list)
df_parsed['bowlers'] = df_parsed.bowlers.map(OrderedDict.fromkeys).apply(list)
df_parsed.head()

,match_id,venue,innings,batting_team,bowling_team,batsmen,bowlers,runs_off_bat,extras,Total_score,...,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]",51,10,61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,335982,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]",19,7,26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,335983,Punjab Cricket Association Stadium,1,Chennai Super Kings,Kings XI Punjab,"[PA Patel, ML Hayden, MEK Hussey]","[B Lee, S Sreesanth, JR Hopes]",50,3,53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,335983,Punjab Cricket Association Stadium,2,Kings XI Punjab,Chennai Super Kings,"[K Goel, JR Hopes]","[JDP Oram, MS Gony]",61,2,63,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,335984,Feroz Shah Kotla,1,Rajasthan Royals,Delhi Capitals,"[T Kohli, YK Pathan, SR Watson, M Kaif]","[GD McGrath, B Geeves, MF Maharoof]",38,2,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
for row,val in enumerate(df_parsed.batsmen):
    for i in range(len(val)):
        df_parsed.loc[row,'bat%s'%(i+1)] = val[i]

for row,val in enumerate(df_parsed.bowlers):
    for i in range(len(val)):
        df_parsed.loc[row,'bow%s'%(i+1)] = val[i]

df_parsed.loc[:,['bat1','bat2','bat3','bat4','bat5','bat6','bat7','bat8','bat9','bat10','bow1','bow2','bow3','bow4','bow5','bow6']]

,bat1,bat2,bat3,bat4,bat5,bat6,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,SC Ganguly,BB McCullum,RT Ponting,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P Kumar,Z Khan,AA Noffke,0.0,0.0,0.0
1,R Dravid,W Jaffer,V Kohli,JH Kallis,CL White,MV Boucher,0.0,0.0,0.0,0.0,AB Dinda,I Sharma,AB Agarkar,0.0,0.0,0.0
2,PA Patel,ML Hayden,MEK Hussey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B Lee,S Sreesanth,JR Hopes,0.0,0.0,0.0
3,K Goel,JR Hopes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JDP Oram,MS Gony,0.0,0.0,0.0,0.0
4,T Kohli,YK Pathan,SR Watson,M Kaif,0.0,0.0,0.0,0.0,0.0,0.0,GD McGrath,B Geeves,MF Maharoof,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,Q de Kock,RG Sharma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DL Chahar,SM Curran,L Ngidi,0.0,0.0,0.0
1449,JC Buttler,YBK Jaiswal,SV Samson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B Kumar,Sandeep Sharma,Rashid Khan,KK Ahmed,0.0,0.0
1450,MK Pandey,JM Bairstow,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Kartik Tyagi,Mustafizur Rahman,C Sakariya,CH Morris,0.0,0.0
1451,P Simran Singh,MA Agarwal,CH Gayle,DJ Malan,0.0,0.0,0.0,0.0,0.0,0.0,I Sharma,MP Stoinis,K Rabada,Avesh Khan,0.0,0.0


In [39]:
if save_files:
    df_parsed.to_csv(Path.joinpath(dataset_path,'4_update_batsmen_bowler_column_names.csv'),index=False)
#df_parsed = pd.read_csv(Path.joinpath(dataset_path, '4_update_batsmen_bowler_column_names.csv'))
df_parsed.head()

,match_id,venue,innings,batting_team,bowling_team,batsmen,bowlers,runs_off_bat,extras,Total_score,...,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]",51,10,61,...,0.0,0.0,0.0,0.0,P Kumar,Z Khan,AA Noffke,0.0,0.0,0.0
1,335982,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]",19,7,26,...,0.0,0.0,0.0,0.0,AB Dinda,I Sharma,AB Agarkar,0.0,0.0,0.0
2,335983,Punjab Cricket Association Stadium,1,Chennai Super Kings,Kings XI Punjab,"[PA Patel, ML Hayden, MEK Hussey]","[B Lee, S Sreesanth, JR Hopes]",50,3,53,...,0.0,0.0,0.0,0.0,B Lee,S Sreesanth,JR Hopes,0.0,0.0,0.0
3,335983,Punjab Cricket Association Stadium,2,Kings XI Punjab,Chennai Super Kings,"[K Goel, JR Hopes]","[JDP Oram, MS Gony]",61,2,63,...,0.0,0.0,0.0,0.0,JDP Oram,MS Gony,0.0,0.0,0.0,0.0
4,335984,Feroz Shah Kotla,1,Rajasthan Royals,Delhi Capitals,"[T Kohli, YK Pathan, SR Watson, M Kaif]","[GD McGrath, B Geeves, MF Maharoof]",38,2,40,...,0.0,0.0,0.0,0.0,GD McGrath,B Geeves,MF Maharoof,0.0,0.0,0.0


In [40]:
df_parsed.head(3)

,match_id,venue,innings,batting_team,bowling_team,batsmen,bowlers,runs_off_bat,extras,Total_score,...,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]",51,10,61,...,0.0,0.0,0.0,0.0,P Kumar,Z Khan,AA Noffke,0.0,0.0,0.0
1,335982,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]",19,7,26,...,0.0,0.0,0.0,0.0,AB Dinda,I Sharma,AB Agarkar,0.0,0.0,0.0
2,335983,Punjab Cricket Association Stadium,1,Chennai Super Kings,Kings XI Punjab,"[PA Patel, ML Hayden, MEK Hussey]","[B Lee, S Sreesanth, JR Hopes]",50,3,53,...,0.0,0.0,0.0,0.0,B Lee,S Sreesanth,JR Hopes,0.0,0.0,0.0


So finally our dataframe is ready with batsmen and bowlers details.

So we can drop few columns which are not important.

In [41]:
df_model = df_parsed[['venue', 'innings', 'batting_team', 'bowling_team', 'bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8','bat9', 'bat10', 'bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6', 'runs_off_bat', 'extras', 'Total_score', 'player_dismissed']]
df_model.columns

Index(['venue', 'innings', 'batting_team', 'bowling_team', 'bat1', 'bat2',
       'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10', 'bow1',
       'bow2', 'bow3', 'bow4', 'bow5', 'bow6', 'runs_off_bat', 'extras',
       'Total_score', 'player_dismissed'],
      dtype='object')

## 4.8: Encode the multiple Dtypes into single Dtype:

Now its time to use, the label encoded values (already done in previous steps) to encode the dataframe.

In [45]:
json_path = Path.joinpath(dataset_path, 'label_encode.json')
with open(json_path) as f:
    data = json.load(f)

condition = False

for col in df_model.columns:
    if col in data.keys():
        condition = True
        col = col
    elif col in ['bat1', 'bat2','bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10']:
        condition = True
        col = 'Players' #'batsmen'
    elif col in ['bow1','bow2', 'bow3', 'bow4', 'bow5', 'bow6']:
        col = 'Players' #'bowlers'
        condition = True

    if condition:
        condition = False
        for key in data[col]:
            df_model = df_model.replace([key], data[col][key])

if save_files:
    df_model.to_csv(Path.joinpath(dataset_path, '5_model_df.csv'), index=False)

In [44]:
df_parsed.head()

,match_id,venue,innings,batting_team,bowling_team,batsmen,bowlers,runs_off_bat,extras,Total_score,...,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]",51,10,61,...,0.0,0.0,0.0,0.0,P Kumar,Z Khan,AA Noffke,0.0,0.0,0.0
1,335982,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]",19,7,26,...,0.0,0.0,0.0,0.0,AB Dinda,I Sharma,AB Agarkar,0.0,0.0,0.0
2,335983,Punjab Cricket Association Stadium,1,Chennai Super Kings,Kings XI Punjab,"[PA Patel, ML Hayden, MEK Hussey]","[B Lee, S Sreesanth, JR Hopes]",50,3,53,...,0.0,0.0,0.0,0.0,B Lee,S Sreesanth,JR Hopes,0.0,0.0,0.0
3,335983,Punjab Cricket Association Stadium,2,Kings XI Punjab,Chennai Super Kings,"[K Goel, JR Hopes]","[JDP Oram, MS Gony]",61,2,63,...,0.0,0.0,0.0,0.0,JDP Oram,MS Gony,0.0,0.0,0.0,0.0
4,335984,Feroz Shah Kotla,1,Rajasthan Royals,Delhi Capitals,"[T Kohli, YK Pathan, SR Watson, M Kaif]","[GD McGrath, B Geeves, MF Maharoof]",38,2,40,...,0.0,0.0,0.0,0.0,GD McGrath,B Geeves,MF Maharoof,0.0,0.0,0.0


So finally dataset is ready. 
Next step is prepare model for training 

# **5. Normalization:**

convert entire dataframe into flot point numbers 

In [ ]:
df_model = df_model.applymap(np.float64)
df_model.info()

As you can see here initially our dataset is of size ~ 30MB, now its in 273 KB

In [ ]:
df_model.iloc[0:4,5:15]

In [ ]:
df_model.corr()

Here I am using **Min Max Scaling** to normalize the dataframe

In [ ]:
df_norm = (df_model - df_model.min())/(df_model.max() - df_model.min())


In [ ]:
df_norm.fillna(0.0,inplace=True)

In [ ]:
df_norm.head() #[['bat9','bat10']]

In [ ]:
df_model.iloc[0:4,4:15]

# **6. Split Train & test data**

Split the dataset into 70% for train and 30% for test
![splitDatasetImg](https://miro.medium.com/max/1400/1*n7Ob33nRMq07BZPbNtfItw.png)

1. Identify the inputs vs targets from the pandas dataframe  and convert into Torch Tensor
2. Create the torch dataset
3. Now split the torch_ds into train_ds and test_ds datasets (70% vs 30%)
4. Create a dataloader for train_ds and test_ds

In [ ]:
inputs = torch.Tensor(df_norm.iloc[:,:-4].values.astype(np.float32))
targets = torch.Tensor(df_norm.loc[:,'Total_score'].values.reshape(-1,1).astype(np.float32))

In [ ]:
torch_ds = torch.utils.data.TensorDataset(inputs,targets)
len(torch_ds)

In [ ]:
train_ds_sz = int(round((len(torch_ds)*split_ratio[0])/100.0))
test_ds_sz  = len(torch_ds) - train_ds_sz

train_ds,test_ds = torch.utils.data.random_split(torch_ds,[train_ds_sz,test_ds_sz])
len(train_ds),len(test_ds)

In [ ]:
train_dl = torch.utils.data.DataLoader(dataset = train_ds,batch_size=train_bs, shuffle = True)
test_dl = torch.utils.data.DataLoader(dataset = test_ds,batch_size=2*train_bs)

# **7. MODEL Selection:**

Here I am using Linear model 

In [ ]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out



model = linearRegression(inputs.shape[1],targets.shape[1])
#model.weight.shape,model.bias.shape

## 7.1 Train & Test:

In [ ]:
from tqdm import tqdm

train_loss, test_loss = [],[]
def train(epoch,model,train_loader,optimizer,loss):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx,(data,target) in enumerate(pbar):
        optimizer.zero_grad()
        output = model(data)
        error = loss(output,target)
        error.backward()
        optimizer.step()
        #if epoch % 10 == 0:
        pbar.set_description(desc= f'Epoch : {epoch} : Train - loss={error.item():.4f} batch_id={batch_idx}')
    train_loss.append(error.item())
    return error.item()
    


def test(epoch,model,test_loader,loss):
    model.eval()
    pbar = tqdm(test_loader)
    with torch.no_grad():
        for batch_idx,(data,target) in enumerate(pbar):
            output = model(data)
            error = loss(output,target)
            #if epoch %10 ==0:
            pbar.set_description(desc= f'Epoch : {epoch} : Test - loss={error.item():.4f} batch_id={batch_idx}')
        test_loss.append(error.item())
        
def saveModel(paramsDict):
    torch.save(paramsDict,Path(Path.cwd(),'model',modelPTH))
    

## 7.2 Loss & optimizer:

In [ ]:
#loss = torch.nn.L1Loss()
loss = torch.nn.MSELoss()
loss_Description = str(loss) # Its declared in Hyper Params section in top

#opt  = torch.optim.SGD(model.parameters(),lr=lr)
#opt  = torch.optim.SGD(model.parameters(),lr=lr,momentum=0.9)
opt  = torch.optim.SGD(model.parameters(),lr=lr,momentum=0.9,nesterov=True) 
optimizer_Description = str(opt)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=opt,mode='min',factor=0.1,patience=10,verbose=True)

print(loss_Description,optimizer_Description)

# **8. Train Model:**

In [ ]:
previous_train_loss = 1e+10

In [ ]:
for epoch in range(epochs):
    present_train_loss = train(epoch,model = model, train_loader = train_dl, optimizer = opt,loss = loss)
    test(epoch,model = model,  test_loader = test_dl, loss=loss)
    scheduler.step(present_train_loss)
    if present_train_loss < previous_train_loss:
        print(f'### Train Loss is reduced in Epoch : {epoch} : {present_train_loss:.4f} < {previous_train_loss:.4f} Train Loss, so saving model')
        previous_train_loss = present_train_loss
        check_point = {'epoch'              : epoch,
                       'model_state_dict'   : model.state_dict(),
                       'optimizer_state_dict': opt.state_dict(),
                       'loss'               : present_train_loss 
                       }
        saveModel(check_point)

## **8.1. Train vs Test Loss Graph:**

In [ ]:
changeDescription   = f'Using : LR Reduce LR On Platue, LR : {lr}, Loss : {str(loss)}, Batch_size : {train_bs}, SGD with Momentum : 0.9, NAG : True'

plt.plot(train_loss)
plt.plot(test_loss);
plt.title('Train vs Test Loss'+' '+changeDescription);
plt.xlabel('epoch');
plt.ylabel('Loss');
plt.legend(['Train_Loss','Test_Loss']);
plt.savefig(pltName);

In [ ]:
shutil.move(pltName,Path.joinpath(Path.cwd(),'snippets',pltName))

In [ ]:
test_inp = torch.Tensor([0.6562, 1.0000, 0.4286, 0.2857, 0.1336, 0.2767, 0.5742, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6360, 0.0511, 0.3145, 0.6621,
        0.0000, 0.0000])

test_op = torch.Tensor([0.4951])
actual    = (test_op*(df_model.Total_score.max() - df_model.Total_score.min() )) + df_model.Total_score.min()
model.eval()
pred     = (model(torch.Tensor(test_inp))*(df_model.Total_score.max() - df_model.Total_score.min() )) + df_model.Total_score.min()
print(f'Predicted Value : {pred.item()} vs Actual value : {actual.item()}')

# **9. Record metrics:**

In [ ]:
metrics = {
    'pltName'       : [pltName],
    'Model'         : ['nn.Linear(20,1)'],
    'BatchSz (Train vs Test)' : [(train_bs, 2*train_bs)],
    'Epochs'        : [epochs],
    'Loss Func'     : [loss_Description],
    'Optimizer'     : [optimizer_Description],
    'LR'            : [lr],
    'Accumilated Train Loss per epoch'  : [list(map(lambda x: float(f'{x:.4f}'), train_loss))],
    'Accumilated Test Loss per epoch'   : [list(map(lambda x: float(f'{x:.4f}'), test_loss))],
    'Min Train Loss': [np.array(list(map(lambda x: float(f'{x:.4f}'), train_loss))).min()],
    'Min Test Loss' : [np.array(list(map(lambda x: float(f'{x:.4f}'), test_loss))).min()],
    'Train Loss Min reached in epoch'   : [np.array(list(map(lambda x: float(f'{x:.4f}'), train_loss))).argmin()],
    'Mean Train Loss': ['{:.4f}'.format(np.array(list(map(lambda x: float(f'{x:.4f}'), train_loss))).mean())],
    'Mean Test Loss' : ['{:.4f}'.format(np.array(list(map(lambda x: float(f'{x:.4f}'), test_loss))).mean())],
    'Test Input'    : [test_inp],
    'Actual Score'  : [actual.item()],
    'Predicted'     : [pred.item()],
    'Notes'         : [changeDescription]

}


In [ ]:
if record:
    if Path.joinpath(Path.cwd(),recordedMetrics).exists():
        df = pd.read_csv(recordedMetrics)
        print('### Concatinating the Metrics to existing Dataframe')
        df_new = pd.concat([df,pd.DataFrame(metrics)],axis=0)
    else:
        print('### Created a new file')
        df_new = pd.DataFrame(metrics)

    df_new.to_csv(Path.joinpath(Path.cwd(),recordedMetrics),index=False)

In [ ]:
df = pd.read_csv(recordedMetrics)
df.head()

# **10. Inference Part:**

   User will enter the prediction details in HTML page, with code we have to parse the user input and encode the string results and then predict

## **10.1 Encode User Data:**

In [ ]:
with open(json_path) as f:
    encode_data = json.load(f)

#encode_data
columns = ['venue','innings','batting_team','bowling_team','bat1','bat2','bat3','bat4','bat5','bat6','bat7','bat8','bat9','bat10','bow1','bow2','bow3','bow4','bow5','bow6']
input_str = ["Dr DY Patil Sports Academy", "1  ", "Kolkata Knight Riders", "Rajasthan Royals", "BB McCullum", "DJ Hussey", "None", "None", "None", "None", "None", "None", "None", "None", "SR Watson", "SK Warne", "SK Warne", "RA Jadeja", "None", "None"]

input_data = dict(zip(columns,input_str))

input_data

In [ ]:
input_encoded = {}
for col in input_data:
    try:
        if col not in ['innings', 'bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10', 'bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6']:
            input_encoded[col] = encode_data[col][input_data[col]]
        elif col == 'innings':
            input_encoded[col] = float(input_data[col].strip())
        elif col in ['bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10', 'bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6']:
            input_encoded[col] = encode_data['Players'][input_data[col]] if input_data[col] != "None" else 0
    except Exception as e:
        print(col)
        
        
print(input_encoded)
    

Normalize the values and feed it to model

In [ ]:
input_norm = {}
for col in input_encoded:
    try:
        if col not in ['innings','bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10', 'bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6']:
            input_norm[col] = (input_encoded[col] - min(encode_data[col].values())) / (max(encode_data[col].values()) - min(encode_data[col].values()))
        elif col in ['bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10', 'bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6']:
            input_norm[col] = (input_encoded[col] - min(encode_data['Players'].values())) / (max(encode_data['Players'].values()) - min(encode_data['Players'].values()))
        elif col == 'innings':
            input_norm[col] = (input_encoded[col] - 1) 
            
            
    except Exception as e:
        print(str(e),input_encoded[col])
    
input_norm

## **10.2 Load Model parameters:**

In [ ]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out


loaded_model = linearRegression(20,1)

print(torch.load(Path.joinpath(Path.cwd(),'model',modelPTH)))

for i in loaded_model.parameters():
    print(i)
    
    
loaded_model.load_state_dict(torch.load(Path.joinpath(Path.cwd(),'model',modelPTH))['model_state_dict'])
loaded_model.eval()

In [ ]:
test_inp = torch.Tensor(list(input_norm.values()))
#test_inp = torch.Tensor([0.6562, 1.0000, 0.4286, 0.2857, 0.1336, 0.2767, 0.5742, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6360, 0.0511, 0.3145, 0.6621,0.0000, 0.0000])

test_op = torch.Tensor([0.4466]) # ([0.4951])
actual    = (test_op*(encode_data['Total_score_max'] - encode_data['Total_score_min'] )) + encode_data['Total_score_min']
pred     = (loaded_model(torch.Tensor(test_inp))*(encode_data['Total_score_max'] - encode_data['Total_score_min'] )) + encode_data['Total_score_min']
print(f'Predicted Value : {pred.item():.0f} vs Actual value : {actual.item():.0f}')

# **11. Visualize Test Results:**

In [ ]:
predicted_values = []
actual_values = []
for input,output in test_dl:
    for row in input:
        pred     = (loaded_model(torch.Tensor(row))*(encode_data['Total_score_max'] - encode_data['Total_score_min'] )) + encode_data['Total_score_min']
        predicted_values.append(pred)
    for op in output:
        actual    = (op*(encode_data['Total_score_max'] - encode_data['Total_score_min'] )) + encode_data['Total_score_min']
        actual_values.append(actual)
    break


In [ ]:
predicted_values[:5]

In [ ]:
actual_values[:5]